In [141]:
import ctypes
from ctypes import wintypes as wt
from ctypes import sizeof
from time import sleep, time
import string

In [142]:
INPUT_MOUSE = 0
INPUT_KEYBOARD = 1
INPUT_HARDWARE = 2

MOUSEEVENTF_ABSOLUTE = 0x8000
MOUSEEVENTF_LEFTDOWN = 0x0002 
MOUSEEVENTF_LEFTUP = 0x0004
MOUSEEVENTF_MOVE = 0x0001
MOUSEEVENTF_RIGHTDOWN = 0x0008
MOUSEEVENTF_RIGHTUP = 0x0010
MOUSEEVENTF_VIRTUALDESK = 0x4000

In [143]:
ULONG_PTR = ctypes.POINTER(ctypes.c_ulong)
LONG = ctypes.c_long

# These depend on whether UNICODE is defined
LPCTSTR = wt.LPCSTR  # LPCWSTR or LPCSTR
LPTSTR = wt.LPSTR  # LPWSTR or LPSTR

WNDENUMPROC = ctypes.WINFUNCTYPE(wt.BOOL, wt.HWND, wt.LPARAM)

In [144]:
class KEYBDINPUT(ctypes.Structure):
    _fields_ = [
        ('wVk', wt.WORD),
        ('wScan', wt.WORD),
        ("dwFlags", wt.DWORD),
        ("time", wt.DWORD),
        ("dwextraInfo", ULONG_PTR)
    ]
    
class MOUSEINPUT(ctypes.Structure):
    _fields_ = [
        ('dx', LONG),
        ('dy', LONG),
        ('mouseData', wt.DWORD),
        ('dwFlags', wt.DWORD),
        ('time', wt.DWORD),
        ('dwExtraInfo', ULONG_PTR),
    ]
    
class HARDWAREINPUT(ctypes.Structure):
    _fields_ = [
        ('uMsg', wt.DWORD),
        ('wParamL', wt.WORD),
        ('wParamH', wt.WORD)
    ]
    

In [145]:
class InptUnion(ctypes.Union):
    _fields_ = [
        ('mi', MOUSEINPUT),
        ('ki', KEYBDINPUT),
        ('hi', HARDWAREINPUT)
    ]

class INPUT(ctypes.Structure):
    _fields_ = [
        ('type', wt.DWORD),
        ('ip', InptUnion)
    ]
    
PINPUT = ctypes.POINTER(INPUT)

In [146]:
KEYEVENTF_KEYUP = 0x0002

def make_kb_input(keycode, scan_code=0, flags=0):
    ip = INPUT()
    ip.type = INPUT_KEYBOARD
    
    ki = KEYBDINPUT()
    ki.wVk = keycode
    ki.wScan = scan_code
    ki.dwFlags = flags
    
    ip.ip.ki = ki
    return ip    

In [148]:
KEYCODES = {}
for i in range(10):
    KEYCODES[str(i)] = 0x30 + i
    
for i, key in enumerate(string.ascii_lowercase):
    KEYCODES[key] = 0x41 + i

In [159]:
# Raw exported windows API functions

user32 = ctypes.windll.user32
k32 = ctypes.windll.kernel32

_SendInput = user32.SendInput
_SendInput.argtypes = [wt.UINT, PINPUT, ctypes.c_int]
_SendInput.restype = wt.UINT

def SendInput(*args):
    res = _SendInput(*args)
    if not res:
        raise OSError(k32.GetLastError())
    return res

FindWindow = user32.FindWindowA
FindWindow.argtypes = [LPCTSTR, LPCTSTR]
FindWindow.restype = wt.HWND

CloseHandle = k32.CloseHandle
CloseHandle.argtypes = [wt.HANDLE]
CloseHandle.restype = wt.BOOL

EnumWindows = user32.EnumWindows
EnumWindows.argtypes = [WNDENUMPROC, wt.LPARAM]
EnumWindows.restype = wt.BOOL

GetWindowText = user32.GetWindowTextA
GetWindowText.argtypes = [wt.HWND, LPTSTR, ctypes.c_int]
GetWindowText.restype = ctypes.c_int

SetActiveWindow = user32.SetActiveWindow
SetActiveWindow.argtypes = [wt.HWND]
SetActiveWindow.restype = wt.HWND

SetForegroundWindow = user32.SetForegroundWindow
SetForegroundWindow.argtypes = [wt.HWND]
SetForegroundWindow.restype = wt.BOOL

GetLastError = k32.GetLastError
GetLastError.argtypes = []
GetLastError.restype = ctypes.c_int

In [160]:
def make_mouse_move(x,y):
    ip = INPUT()
    ip.type = INPUT_MOUSE
    
    mi = MOUSEINPUT()
    mi.dx = x
    mi.dy = y
    mi.mouseData = 0
    mi.time = 0
    mi.dwFlags=(MOUSEEVENTF_MOVE) | MOUSEEVENTF_ABSOLUTE | MOUSEEVENTF_VIRTUALDESK
    ip.ip.mi = mi
    return ip

In [161]:
def make_mouse_left_click(dwFlags):
    ip = INPUT()
    ip.type = INPUT_MOUSE
    
    mi = MOUSEINPUT()
    mi.dx = 0
    mi.dy = 0
    mi.mouseData = 0
    mi.time = 0
    mi.dwFlags = MOUSEEVENTF_ABSOLUTE | MOUSEEVENTF_VIRTUALDESK | dwFlags
    ip.ip.mi = mi
    return ip

In [162]:
def left_click(x,y):
    ip1 = INPUT()
    ip1.type = INPUT_MOUSE
    
    mi = MOUSEINPUT()
    mi.dx = x
    mi.dy = y
    mi.mouseData = mi.time = 0
    mi.dwFlags = MOUSEEVENTF_ABSOLUTE | MOUSEEVENTF_VIRTUALDESK | MOUSEEVENTF_MOVE | MOUSEEVENTF_LEFTDOWN
    ip1.ip.mi = mi
    
    ip2 = INPUT()
    ip2.type = INPUT_MOUSE
    
    mi = MOUSEINPUT()
    mi.dx = 0
    mi.dy = 0
    mi.mouseData = mi.time = 0
    mi.dwFlags = MOUSEEVENTF_ABSOLUTE | MOUSEEVENTF_VIRTUALDESK | MOUSEEVENTF_LEFTUP
    ip2.ip.mi = mi
    
    sz = ctypes.sizeof(ip1)
    arr = (INPUT * 2)(ip1, ip2)
    return SendInput(2, arr, sz)

In [163]:
import win32api
SCREEN_X = win32api.GetSystemMetrics(78)
SCREEN_Y = win32api.GetSystemMetrics(79)

def norm(p):
    return int(p.x / (SCREEN_X) * 65535), int(p.y / (SCREEN_Y) * 65536)

def norm2(p):
    return int(p[0] / (SCREEN_X) * 65535), int(p[1] / (SCREEN_Y) * 65536)

In [171]:
def send_keyboard_input(keycode, scan_code=0):
    ip1 = make_kb_input(keycode, scan_code, 0)
    ip2 = make_kb_input(keycode, scan_code, KEYEVENTF_KEYUP)
    return send_inputs(ip1, ip2)

def send_key(key):
    vk = KEYCODES[key]
    send_keyboard_input(vk, 0)

def mouse_move(x,y):
    ip = make_mouse_move(x,y)
    return SendInput(1, PINPUT(ip), ctypes.sizeof(ip))
    
def mouse_click():
    ip2 = make_mouse_left_click(MOUSEEVENTF_LEFTDOWN)
    ip3 = make_mouse_left_click(MOUSEEVENTF_LEFTUP)
    send_inputs(ip2, ip3)

def send_inputs(*ips):
    sz = ctypes.sizeof(ips[0])
    array = (INPUT * len(ips))(*ips)
    return SendInput(len(ips), array, sz)

In [17]:
def get_pos():
    point = ctypes.wintypes.POINT()
    ctypes.windll.user32.GetCursorPos(ctypes.byref(point))
    x, y = norm(point)
    return x,y

def get_pos2():
    point = ctypes.wintypes.POINT()
    ctypes.windll.user32.GetCursorPos(ctypes.byref(point))
    return point.x, point.y

In [18]:
def clickfind():
    down = False
    while True:
        try:
            sleep(0.01)
            print("\r    (%d, %d),"%get_pos(), end="")
            if win32api.GetKeyState(0xC0) & 0x8000:
                if not down:
                    print()
                    down = True
            else:
                down = False
        except:
            break

In [19]:
def right_click_down():
    return win32api.GetKeyState(0x02) & 0x8000

def wait(s):
    end = time() + s
    while True:
        left = end - time()
        if left > 0:
            sleep(min(left, 0.1))
        else:
            break


In [52]:
# If screen resolution affects these (it probably does), then I need a 3 point calibration to 
# scale all coordinates...

OFFSET_X = 0
OFFSET_Y = 0
REF_X = 8450
REF_Y = 20607

def get_offset():
    global OFFSET_X, OFFSET_Y
    while True:
        if right_click_down():
            x,y = get_pos()
            OFFSET_X = x - REF_X
            OFFSET_Y = y - REF_Y
            break
    sleep(0.01)
    
def translate_coord(x,y):
    return x + OFFSET_X, y + OFFSET_Y
    
BUTTON_HEIGHT = 2157

In [41]:
def find_btn_coords(features):
    coords = []
    for f in features:
        print("(%r, "%f, end="")
        while True:
            if right_click_down():
                if not down:
                    down = True
                    p = get_pos()
                    coords.append(p)
                    print("(%d, %d)),"%p)
                    break
            else:
                down = False

In [120]:
# features = [
#     "basic_training",
#     "fight_boss",
#     "money_pit",
#     "adventure",
#     "inventory",
#     "augmentation", 
#     "advanced_training",
#     "time_machine",
#     "blood_magic"
# ]

# find_btn_coords(features)
# basic_training = []
# for attack in "idle", "regular", "strong", "parry", "piercing", "ultimate":
#     for suffix in "_plus", "_minus", "_cap":
#         basic_training.append(attack+suffix)
# find_btn_coords(basic_training)

# fight_boss = ["nuke", "stop", "fight"]
# find_btn_coords(fight_boss)

# augments = []
# for btn in "safety_scissors", "danger_scissors", "milk_infusion", "drinking_milk", "cannon_implant", "missile_launcher":
#     for suffix in "_plus", "_minus":
#         augments.append(btn+suffix)
# find_btn_coords(augments)

# blood_magic = []
# for btn in "tack", "papercuts", "hickey", "barbed_wire", "blood_bank", "decapitation", "woodchipper", "inside_out":
#     for suffix in "_plus", "_minus", "_cap":
#         blood_magic.append(btn+suffix)
# find_btn_coords(blood_magic)

# rebirth = ["rebirth", "yes", "no"]
# find_btn_coords(rebirth)

# adventure = ["safe", "tutorial", "sewers", "forest", "cave_of_many_things"]
# find_btn_coords(adventure)

('safe', (20630, 34661)),
('tutorial', (20565, 36044)),
('sewers', (20350, 37355)),
('forest', (20220, 39103)),
('cave_of_many_things', (20328, 40413)),


In [129]:
FEATURE_BUTTONS = [
    ('basic_training', (8407, 20753)),
    ('fight_boss', (8429, 22791)),
    ('money_pit', (8515, 25122)),
    ('adventure', (8429, 27161)),
    ('inventory', (8429, 29272)),
    ('augmentation', (8429, 31530)),
    ('advanced_training', (8493, 33787)),
    ('time_machine', (8450, 35826)),
    ('blood_magic', (8429, 38010)),
    ('rebirth', (5281, 47477))
]

BASIC_TRAINING = [
    ('idle_plus', (21557, 27452)),
    ('idle_minus', (22290, 27525)),
    ('idle_cap', (23109, 27597)),
    ('regular_plus', (21557, 29928)),
    ('regular_minus', (22355, 30146)),
    ('regular_cap', (23174, 29855)),
    ('strong_plus', (21579, 32403)),
    ('strong_minus', (22268, 32403)),
    ('strong_cap', (23131, 32476)),
    ('parry_plus', (21557, 35098)),
    ('parry_minus', (22398, 35170)),
    ('parry_cap', (23088, 35025)),
    ('piercing_plus', (21514, 37355)),
    ('piercing_minus', (22398, 37355)),
    ('piercing_cap', (23066, 37209)),
    ('ultimate_plus', (21557, 39612)),
    ('ultimate_minus', (22290, 39904)),
    ('ultimate_cap', (23109, 39831)),
]

FIGHT_BOSS = [
    ('nuke', (16879, 25194)),
    ('stop', (16814, 29199)),
    ('fight', (16901, 33204)),
]

AUGMENTS = [
    ('safety_scissors_plus', (15068, 36263)),
    ('safety_scissors_minus', (15758, 36408)),
    ('danger_scissors_plus', (15047, 38520)),
    ('danger_scissors_minus', (15801, 38374)),
    ('milk_infusion_plus', (15047, 40996)),
    ('milk_infusion_minus', (15801, 40996)),
    ('drinking_milk_plus', (15068, 43035)),
    ('drinking_milk_minus', (15801, 43326)),
    ('cannon_implant_plus', (15111, 45875)),
    ('cannon_implant_minus', (15801, 45802)),
    ('missile_launcher_plus', (15090, 47914)),
    ('missile_launcher_minus', (15801, 47841)),
]

BLOOD_MAGIC = [
    ('tack_plus', (14120, 33641)),
    ('tack_minus', (14917, 33641)),
    ('tack_cap', (15672, 33641)),
    ('papercuts_plus', (14141, 36263)),
    ('papercuts_minus', (14853, 36263)),
    ('papercuts_cap', (15693, 36263)),
    ('hickey_plus', (14141, 38884)),
    ('hickey_minus', (14917, 38957)),
    ('hickey_cap', (15715, 38811)),
    ('barbed_wire_plus', (14120, 41360)),
    ('barbed_wire_minus', (14896, 41287)),
    ('barbed_wire_cap', (15693, 41287)),
    ('blood_bank_plus', (14098, 43909)),
    ('blood_bank_minus', (14896, 43981)),
    ('blood_bank_cap', (15715, 44054)),
    ('decapitation_plus', (14120, 46384)),
    ('decapitation_minus', (14874, 46312)),
    ('decapitation_cap', (15715, 46530)),
    ('woodchipper_plus', (14120, 48933)),
    ('woodchipper_minus', (14853, 49006)),
    ('woodchipper_cap', (15693, 49079)),
    ('inside_out_plus', (14141, 51409)),
    ('inside_out_minus', (14874, 51482)),
    ('inside_out_cap', (15693, 51482)),
]

ADVENTURE_ZONES = [
    ('safe', (20630, 34661)),
    ('tutorial', (20565, 36044)),
    ('sewers', (20350, 37355)),
    ('forest', (20220, 39103)),
    ('cave_of_many_things', (20328, 40413)),
]

ADVENTURE_ZONE_MENU = (20415, 32403)
ADVENTURE = [
    
]

REBIRTH = [
    ('rebirth', (15219, 55195)),
    ('yes', (12805, 40413)),
    ('no', (14529, 40268)),
]

SIMPLE_MENUS = [
    ('basic_training', BASIC_TRAINING),
    ('fight_boss', FIGHT_BOSS),
    ('augmentation', AUGMENTS),
    ('blood_magic', BLOOD_MAGIC),
    ('rebirth', REBIRTH),
    #('adventure', ADVENTURE)
]

In [180]:
import weakref

class SimpleItem():
    def __init__(self, parent, label, x, y):
        if parent is None:
            self._parent = None
        else:
            self._parent = weakref.ref(parent)
        self.label = label
        self.x = x
        self.y = y
        self.items = {}
        
    @property
    def parent(self):
        if self._parent is None:
            p = self
        else:
            p = self._parent()
            if p is None:
                p = self
        return p
    
    def click(self):
        x, y = translate_coord(self.x, self.y)
        left_click(x, y)
        
    def add_item(self, label, item):
        if label in self.items:
            raise ValueError("Item '%s' already exists in '%s'"%(label, self.get_full_name()))
        self.items[label] = item
            
    def get_full_name(self):
        name = self.label
        p = self
        while True:
            if p is None or p.parent is p:
                break
            p = p.parent
            name = p.label + "." + name
        return name
    
    def lookup(self, key):
        parts = key.split(".")
        item = self
        for p in parts:
            item = item.items.get(p)
            if item is None:
                raise KeyError(key)
        return item
    
    def click_sequence(self, *tags, st=0.1):
        it = self
        for t in tags:
            it = it.lookup(t)
            it.click()
            sleep(st)
    
    get = __getitem__ = lookup  # alias

class BasicButton(SimpleItem):
    pass

class SimpleMenu(SimpleItem):
        
    def add_button(self, label, x, y):
        btn = BasicButton(self, label, x, y)
        self.add_item(label, btn)
        return btn
        
    def add_menu(self, label, x, y):
        menu = SimpleMenu(self, label, x, y)
        self.add_item(label, menu)
        return menu
        
    def add_menu_with_buttons(self, label, x, y, buttons):
        menu = SimpleMenu(self, label, x, y)
        self.add_item(label, menu)
        for b, (x,y) in buttons:
            menu.add_button(b, x, y)
        return menu


In [187]:
class TrainingInfo():
    def __init__(self, start_time=None):
        start_time = start_time or time()
        self.level = ""
#         self.eps = eps
        self.start_time = start_time
        self.current_level_time = start_time
        self.thresholds = [
            ('idle', 0),
            ('regular', 5000),
            ('strong', 10000),
            ('parry', 15000),
            ('piercing', 20000),
            ('ultimate', 25000)
        ]
        self.idx = -1
        
    def can_train_next(self):
        if self.idx == len(self.thresholds) - 1:
            return ""
        elapsed = time() - self.current_level_time - 1
        levels = elapsed * 50 
        lvl, thresh = self.thresholds[self.idx + 1]
        if levels > thresh:
            return lvl
        return ""
    
    def energy_capped_for_current(self):
        # Todo: calculate current energy & required cap energy
        return True  
                
    def advance(self):
        if self.idx == len(self.thresholds) - 1:
            return
        self.idx += 1
        self.current_level_time = time()
        self.level = self.thresholds[self.idx][0]

class Game(SimpleMenu):
    def __init__(self, stats):
        super().__init__(None, "Game", 0, 0)
        self.current_menu = ""
        self.stats = stats
        self.training = TrainingInfo()
        self.start_time = time()
        self.current_adventure = ""
        
    def run(self, start_time=None):
        self.start_time = start_time or time()
        
        # new game actions
        # TODO: intelligently calculate whether we actually have
        # enough energy to cap training, rather than this hardcoded wait
        wait(2)
        self.train()
        wait(2)
        self.nuke()
        self.adventure()
        
        while True:
            if right_click_down():
                return  # abort
            self.train()
            self.adventure()
            self.spend_idle_energy()
            self.spend_idle_magic()
            sleep(0.1)
            
    def spend_idle_magic(self):
        best_spell = "tack"
        self.click_sequence("blood_magic", best_spell + "_plus")
            
    def spend_idle_energy(self):
        if not self.training.energy_capped_for_current():
            self.click_sequence("basic_training", self.training.level + "_cap")
        else:
            # use energy on best ROI for total dps
            self.click_sequence("augmentation", "safety_scissors_plus")
            
    def nuke(self):
        self.click_sequence("fight_boss", "nuke")
        
    def select_best_idle_adventure(self):
        return 'forest'
        
    def adventure(self):
        adv = self.select_best_idle_adventure()
        if adv != self.current_adventure:
            self.click_sequence("adventure", "adventure_zones", adv)
            self.current_adventure = adv
        
    def begin(self):
        #self.click_sequence("rebirth", "rebirth.rebirth", "rebirth.yes")
        self.start_time = time()
        
    def current_time(self):
        return time() - self.start_time
    
    def train(self):
        lvl = self.training.can_train_next()
        if lvl:
            send_key('r')
            sleep(0.1)
            self.click_sequence("basic_training", lvl + "_cap")
            self.training.advance()
        

In [188]:
stats = {
    'energy': {
        'base_power': 4.4,
        'equip_power_modifier': 1.37,
        'bars': 5,
        'equip_bars_modifier': 1,
        'cap': 120000,
        'equip_cap_modifier': 1,
        'base_speed': 32.5,
        'equip_speed_modifier': 1.54
    },
    'magic': {
        'base_power': 1,
        'equip_power_modifier': 1.00,
        'bars': 1,
        'equip_bars_modifier': 1,
        'cap': 10000,
        'equip_cap_modifier': 1,
        'base_speed': 3.3,
        'equip_speed_modifier': 1.00
    }
}

game = Game(stats)
FB = {l:c for l,c in FEATURE_BUTTONS}
for lbl, buttons in SIMPLE_MENUS:
    x, y = FB[lbl]
    game.add_menu_with_buttons(lbl, x, y, buttons)
    
# adventure
x, y = FB['adventure']
adv = game.add_menu('adventure', x, y)
x, y = ADVENTURE_ZONE_MENU
az = adv.add_menu('adventure_zones', x, y)
for b, (x,y) in ADVENTURE_ZONES:
    az.add_button(b, x, y)

In [189]:
game.run(start_time = time()-3600)